## Example for ImArray
ImArray is a software to image an object in the sky by an array of ground Telescopes
Copyright (C) 2016  Thomas Vuillaume

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>

The author may be contacted @
thomas.vuillaume@lapp.in2p3.fr

Let's start with some imports

In [1]:
import geometry as geo
import numpy as np
import matplotlib.pyplot as plt
import time
import math
import object as obj
import CameraImage as ci
import Hillas as hillas
import os
import vizualisation as viz

We can define some shower parameters:

In [2]:
impact_point = np.array([0, 10, 0])

# shower direction
salt = math.radians(74)
saz = math.radians(0)

# pointing direction
talt = math.radians(74)
taz = math.radians(0)

# shower parameters
shower_top = 15000
shower_length = 12000
shower_width = 200

#number of points in the shower
npoints = 50000

Here, our shower is a geometrical object (an ellpisoid in this case) randomly filled with points:

In [3]:
shower = obj.random_ellipsoide(shower_top, shower_length, shower_width, salt, saz, impact_point, npoints)

Let's now define a telescopes array configuration:

In [20]:
tel_normal = geo.altaz_to_normal(talt, taz)

tel1 = geo.Telescope([500, 100, -100], tel_normal, 0)
tel2 = geo.Telescope([-10, 300, 100], tel_normal, 0)
tel3 = geo.Telescope([-500, 400, 10], tel_normal, 0)
tel4 = geo.Telescope([-350, -500, 0], tel_normal, 0)
tel5 = geo.Telescope([600, -100, 150], tel_normal, 0)
tel6 = geo.Telescope([300, -300, 50], tel_normal, 0)
tel7 = geo.Telescope([80, 0, 0], tel_normal, 0)
tel8 = geo.Telescope([-80, 0, 0], tel_normal, 0)
tel9 = geo.Telescope([0, -80, 0], tel_normal, 0)
tel10 = geo.Telescope([0, 80, 0], tel_normal, 0)

alltel = [tel1, tel2, tel3, tel4, tel5, tel6, tel7, tel8, tel9, tel10]

The telecopes positions can also be load from a data file with:

In [21]:
#alltel = geo.load_telescopes("data/tel_pos.dat")

We can check that no pair of telescopes are set at the same location:

In [22]:
geo.telescopes_unicity(alltel)

True

Let's vizualise what we have done so far:

In [23]:
viz.plot_shower3d(shower,alltel)

In [24]:
triggered_telescopes = []
trigger_intensity = 0.

noise = 0
HP = []

In [25]:
for tel in alltel:
    shower_image = geo.image_shower_pfo(shower, tel)
    shower_cam = geo.site_to_camera_cartesian(shower_image, tel)
    pixels_signal = ci.camera_image(tel, shower_cam[:, [0, 1]], noise)

    if len(pixels_signal.nonzero()[0]) > 1 and pixels_signal.sum() > trigger_intensity:
        hp = hillas.hillas_parameters(tel.pixel_tab[:, 0], tel.pixel_tab[:, 1], pixels_signal)
        HP.append(hp)
        triggered_telescopes.append(tel)


In [29]:
pa = None
p = None

One can now compute the impact parameter with two Hillas reconstruction:

In [30]:
pa = hillas.impact_parameter_average(triggered_telescopes, HP, talt, taz)
p = hillas.impact_parameter_ponderated(triggered_telescopes, HP, talt, taz)

And compare the results:

In [31]:
print("Real impact parameter : ", impact_point)
print("Reconstruction with simple average = %s \tError = %.2fm" % (pa, math.sqrt(((impact_point-pa)**2).sum())))
print("Reconstruction with ponderation and cut = %s \tError = %.2fm" % (p, math.sqrt(((impact_point-p)**2).sum())))

Real impact parameter :  [ 0 10  0]
Reconstruction with simple average = [4.0699870494689199, 45.993253615444083, 0.0] 	Error = 36.22m
Reconstruction with ponderation and cut = [-5.5482795886712806, 4.9422180930660113, 0.0] 	Error = 7.51m


In [129]:
shower_image = geo.image_shower_pfo_2(shower, alltel[0])
plt.scatter(shower_image[:,0], shower_image[:,1])
plt.show()

In [130]:
shower_cam = geo.site_to_camera_cartesian_2(shower_image, alltel[0])
plt.scatter(shower_cam[:,0], shower_cam[:,1])
plt.show()

In [131]:
pixels_signal = ci.camera_image(alltel[0], shower_cam[:, [0, 1]], noise)
#for point in shower_image:
#    print(geo.is_point_in_camera_plane(point, tel))
    
plt.scatter(tel.pixel_tab[:,0],tel.pixel_tab[:,1],c= pixels_signal)
plt.axis('equal')
plt.show()